In [1]:
%load_ext autoreload
%autoreload 2

import os, json
import numpy as np
import pandas as pd

import torch
import pytorch_lightning as pl

from ablang_train import ABtokenizer, AbLang, TrainingFrame, CallbackHandler, AbDataModule, ablang_parse_args
from ablang_train import models

/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/neptune/internal/backends/hosted_client.py:50: NeptuneDeprecationWarning: The 'neptune-client' package has been deprecated and will be removed in the future. Install the 'neptune' package instead. For more, see https://docs.neptune.ai/setup/upgrading/
  from neptune.version import version as neptune_client_version
/data/iraqbabbler/olsen/miniconda3/envs/ablang-train/lib/python3.9/site-packages/pytorch_lightning/loggers/neptune.py:40: NeptuneDeprecationWarning: You're importing the Neptune client library via the deprecated `neptune.new` module, which will be removed in a future release. Import directly from `neptune` instead.
  from neptune import new as neptune


In [2]:
# SET ARGUMENTS AND HPARAMS
arguments = ablang_parse_args(args=["--json_args", "../reports/model_arguments/05_paired_updataFineTune.json"])
#arguments.model_specific_args.data_path = '/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/'# '../data/single_data/' #
#arguments.model_specific_args.eval_path = '/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/'
#arguments.model_specific_args.use_moe = False
#arguments.model_specific_args.mask_technique = 'mix'
#arguments.model_specific_args.change_percent = -1
#arguments.model_specific_args.over_sample_data = 1
#arguments.model_specific_args.variable_masking = True
#arguments.model_specific_args.path_start_weights = "/vols/bitbucket/olsen/models/ABLANG-529/"
arguments.model_specific_args

Namespace(name='paired_updataFT', json_args='../reports/model_arguments/05_paired_updataFineTune.json', n_encoder_blocks=6, hidden_embed_size=320, n_attn_heads=20, dropout=0.0, use_tkn_dropout=False, loss_fn='Focal_Loss', a_fn='swiglu', fl_gamma=2, use_moe=False, mask_percent=0.25, variable_masking=True, mask_technique='mix', change_percent=0.1, leave_percent=0.1, initializer_range=0.02, layer_norm_eps=1e-12, data_path='/data/nagagpu03/not-backed-up/nvme00/olsen/oas_data/nov2022-paired-all/', out_path='/data/iraqbabbler/olsen/Documents/projects/AbLang/model-catalogue/paired-ablang/train_ablang_pair/reports/models', eval_path='/vols/bitbucket/olsen/processed_oas_data/nov2022/nov2022-paired-all/', path_start_weights='/vols/bitbucket/olsen/models/ABLANG-529/', cpus=1, max_fit_batch_size=100, effective_batch_size=8192, num_training_steps=1000, warmup_steps=100, learning_rate=0.0001, cdr3_focus=1, weight_decay=0.01, adam_epsilon=1e-08, adam_betas=[0.9, 0.98], seed=42, eval_batch_size=100, o

In [3]:
callbacks = CallbackHandler(
    save_step_frequency=arguments.model_specific_args.log_every_n_steps, 
    progress_refresh_rate=0, 
    outpath=arguments.model_specific_args.out_path
)

In [4]:
def enforce_reproducibility(seed=42):
    # Sets seed manually for both CPU and CUDA
    torch.manual_seed(seed)
    # For atomic operations there is currently
    # no simple way to enforce determinism, as
    # the order of parallel operations is not known.
    #
    # CUDNN
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False
    # System based
    np.random.seed(seed)
    
    pl.seed_everything(seed)
    
# SET SEED - IMPORTANT FOR MULTIPLE GPUS, OTHERWISE GOOD FOR REPRODUCIBILITY
enforce_reproducibility(arguments.model_specific_args.seed)

Global seed set to 42


In [5]:
# LOAD AND INITIATE DATA

ablang_dm = AbDataModule(arguments.model_specific_args, ABtokenizer) 
# You are supposed to just be able to add abrep to the fit function, but it doesn't work when using multiple GPUs
ablang_dm.setup('fit')

train = ablang_dm.train_dataloader()
val = ablang_dm.val_dataloader()

ValueError: num_samples should be a positive integer value, but got num_samples=0

In [ ]:
for batch in train:
    
    print(batch['input'][0])
    print(batch['labels'][:len(batch['input'][0])])
    
    print((batch['labels'][:len(batch['input'][0])] != -100).sum())
    
    break

In [ ]:
arguments.model_specific_args

In [ ]:
# LOAD MODEL
model = TrainingFrame(arguments.model_specific_args, AbLang, ABtokenizer)

# INITIALISE TRAINER
trainer = pl.Trainer(**arguments.trainer_args, callbacks=callbacks())

In [ ]:
%%time
trainer.fit(model, train_dataloaders=train, val_dataloaders=val)

In [ ]:
seq_to_restore = 'EVQLVESGPGLVQPGKSLRLSCVASGFTFSGYGMHWVRQAPGKGLEWIALIIYDESNKYYADSVKGRFTISRDNSKNTLYLQMSSLRAEDTAVFYCAKVKFYDPTAPNDYWGQGTLVTVSS|'

In [ ]:
model1 = model.ablang
tokenizer1 = model.tokenizer

with torch.no_grad():
    tokenPreds = model1(tokenizer1([seq_to_restore], pad=True, w_extra_tkns=False, device='cpu'))

tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)

aaPreds = tokenizer1(tokenMAX.indices, w_extra_tkns=False, mode='decode', device='cpu')

unkMatrix = torch.zeros(tokenMAX[0].shape, dtype=torch.long, device='cpu') + 21

aaPreds50 = ['-'.join(tokenizer1(torch.where(tokenMAX[0]<=.5, unkMatrix, tokenMAX[1]).detach(), w_extra_tkns=False, mode='decode')[0].split('<unk>'))]

In [ ]:
tokenPreds[0][2]

In [ ]:
tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)
tokenMAX[1]

In [ ]:
masked_to_restore = '***LVESGPGLVQPGKSLRLSCVASGFTFSGYGMHWVRQAPGKGLEWIALIIYDESNKYYADSVKGRFTISRDNSKNTLYLQMSSLRAEDTAVFYCAKVKFYDPTAPND************'
masked_to_restore = '<********************************GMHWVRQAPGKGLEWIALI*********ADSVKGRFTISRDNSKNTLYLQMSSLRA*********************************>|'

In [ ]:
with torch.no_grad():
    tokenPreds = model1(tokenizer1([masked_to_restore], pad=True, w_extra_tkns=False, device='cpu'))
    tokenMAX = torch.max(torch.nn.Softmax(dim=-1)(tokenPreds), -1)
    masked_aaPreds = tokenizer1(tokenMAX[1], mode='decode')
masked_aaPreds

In [ ]:
print(aaPreds[0])
print(aaPreds50[0])
print(seq_to_restore)